In [1]:
from classy import Class
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
import scipy.integrate as sc
import scipy.optimize as optimize
import scipy.interpolate

import matplotlib.pyplot as plt
import scipy.integrate as inv

import scipy.special as special
from time import time

In [8]:

def TEST(kvec=[1],k_ind=0,z=800,
         mv_test=1 ,Omega_M = 0.99,
         Omega_b=0.01,PLOT_FIGUR=True,
         EdSitter=True,interpolation_type="cubic"):

    if EdSitter==True:
        Omega_M = 0.95
    Omega_lambda = 1 - Omega_M - Omega_b
    cs_factor = (25*special.zeta(5)/special.zeta(3))**0.5
    cs_factor /= 4
    factor = cs_factor
    k = kvec[k_ind]
    
    # CLASS stuff
    H0=67
    Tv_k = 1.947
    # in MeV
    h = 0.6704
    kb = 8.617e-11 # MeV/K
    T0 = Tv_k * kb
    cosmo_dict ={'H0':H0, 'Omega_cdm':Omega_M,'Omega_b':Omega_b, 'k_output_values':str(kvec).strip('[]')
                 #,"N_ncdm":1,"m_ncdm":mv_test # massen er givet i eV s' jeg ganger min egen test m_v skal være en faktor  1e-6 mindre
               ,'output':'tCl, lCl, dTk, vTk, pCl', 'lensing':'yes',
                "ncdm_fluid_approximation":3
                , "YHe":0.25
                }

    c = 3*1e5 # speed of lign in km/s
    H0 /= c # i 1/Mpc

    cosmo = Class()
    cosmo.set(cosmo_dict)
    cosmo.compute()
    bg = cosmo.get_background()
    tid = bg['proper time [Gyr]'] * c / 306.4 # det kommer af Gyrs * c = 306.4 Mpc
    pts = cosmo.get_perturbations()['scalar']
    # Here we create/import the powerspectrum values from class
    tranf = cosmo.get_transfer(z=0)
    prim = cosmo.get_primordial()
    k = tranf['k (h/Mpc)'] * h
    
    d_m_af_k =  scipy.interpolate.interp1d(k,tranf["d_m"])
    prim_interp = scipy.interpolate.interp1d(prim["k [1/Mpc]"],prim["P_scalar(k)"], fill_value="extrapolate",kind="cubic")
    p = lambda k: 2*np.pi**2 * d_m_af_k(k)** 2 *  prim_interp(k) * h**3  *k**(-3)
    
    
    mv = mv_test * 1e-6

    def cs(a):
        Tv_k = 1.947 # kelvin # MeV
        kb = 8.617e-11 # MeV/K
        T0 = Tv_k * kb
        mv = mv_test * 1e-6
        cs_factor = (25*special.zeta(5)/special.zeta(3))**0.5
        cs_factor /= 4
        return cs_factor *T0 / a / mv

    #print(f"{cs(1)=}")
    
    
    def H(a):
        return H0 * np.sqrt(Omega_M /a + Omega_lambda * a**2)
    
    
    # henter class data
    #=======================
    delta_test      = np.empty(len(kvec))
    delta_M      = np.empty(len(kvec))
    delta_approx = np.empty(len(kvec))
    b_list = np.empty(len(kvec))
    
    # her genere 
    for kind,k in enumerate(kvec): 
        delta_class = pts[kind]["delta_cdm"]
        theta_class = pts[kind]["theta_cdm"]
        tau_class = pts[kind]["tau [Mpc]"]
        a_class = pts[kind]["a"]
        interp_tau_from_a = scipy.interpolate.interp1d(a_class,tau_class,kind=interpolation_type)
        
        delta_interp = scipy.interpolate.interp1d(tau_class,delta_class,kind=interpolation_type)
        a_interp = scipy.interpolate.interp1d(tau_class,a_class,kind=interpolation_type)
        a0 = 1/(1+z)
        a_span = [a0,1]
        T0 = float(interp_tau_from_a(a0))
        #T_0 = inter_tid_from_a
        delta0 = delta_interp(T0)
        t0 = tau_class[-1]
        tspan = [T0,t0]
        t = np.logspace(np.log10(T0),np.log10(t0),100)
        def Dfunc(tau,y):
            a = a_interp(tau)
            D = y[0]
            dD = y[1]
            d2D = - H(a) * dD + (3/2) * H0*H0 * Omega_M * D / a
            return [dD,d2D]

        delta_diff = lambda t,h: (delta_interp(t+h)-delta_interp(t))/h
        start0 = [delta_interp(T0),delta_diff(T0,1)]
        sol = scipy.integrate.solve_ivp(Dfunc, t_span = tspan, y0 = start0)
    
        delta_M[kind] = sol.y[0][-1]
        delta_approx[kind] = delta0/a0 * 1
        delta_test[kind] = delta_class[-1]
        
    plt.loglog(sol.t,-sol.y[0])
    plt.loglog(t_class,-delta_class)
    
    

    def f2(t):
        C1 = np.sqrt(neu_tid(T_0)**2 + (theta_neu_tid(T_0)*a0/ (cs(a0)*k))**2)
        print(f"{theta_neu_tid(T_0)=}")
        C2 = np.pi / 2 * ( 3 * cs(a0) * k * t0 * a0 )**2 * (delta0/a0)
        dcdm = delta0/a0        
        a = (t/t0)**(2/3)
        val = 3 * cs(a) * k * t0 * np.sqrt(a)        
        return dcdm*a+C1*Cos(val)+val**2*a*dcdm*Cos(val)*CosIntegral(val)-C2*Sin(val)+val**2*a*dcdm*Sin(val)*SinIntegral(val) 

    
    plt.loglog()
    
    
    delta_app =  scipy.interpolate.interp1d(kvec,delta_approx)
    p_approx = lambda k: 2*np.pi**2 * delta_app(k)** 2 *  prim_interp(k) * h**3  *k**(-3)    
    test =  scipy.interpolate.interp1d(kvec,delta_test)
    p_test = lambda k: 2*np.pi**2 * test(k)** 2 *  prim_interp(k) * h**3  *k**(-3)
    delta =  scipy.interpolate.interp1d(kvec,delta_M)
    p_min = lambda k: 2*np.pi**2 * delta(k)** 2 *  prim_interp(k) * h**3  *k**(-3)    
    
    k =np.linspace(kvec[0],kvec[-1],100)
    
    plt.figure(np.random.randint(1,324234))
    
    plt.loglog(k,p(k),label="class")
    plt.loglog(k,p_approx(k),label="approx.")
    plt.loglog(k,p_min(k),label="sim")
    plt.loglog(k,p_test(k),label="test")
    plt.legend()
    
    def K(k1,t,theta):
        k2 = k1 * t
        bn = 1
        an = 2/7
        res= (bn - an) + bn/2 *  np.cos(np.pi-theta) * (k2/k1 + k1/k2) + an * (np.cos(np.pi-theta))**2
        return res 
    
    def reduceret_B_spektrum(k1,t,theta):
        k2 = t * k1
        phi = np.pi - theta
        k3 = np.sqrt((k2*np.sin(phi))**2 + (k2*np.cos(phi)+k1)**2)    
        theta2 = np.arccos((k3**2 + k2**2 - k1**2) / ( 2 * k3 * k2 )) 
        theta3 = np.pi - theta2 - theta
        t2 = k3/k2
        t3 = k1/k3
        res =  2 *  (p(k1)*p(k2) * K(k1,t,theta)
                    +p(k3)*p(k1) * K(k3,t3,theta3) 
                    +p(k2)*p(k3) * K(k2,t2,theta2)) / (p(k1)*p(k2)+p(k3)*p(k1)+p(k2)*p(k3))
        return res
      
    
    def Q_min(k1,t,theta):
        k2 = t * k1
        phi = np.pi - theta
        k3 = np.sqrt((k2*np.sin(phi))**2 + (k2*np.cos(phi)+k1)**2)    
        theta2 = np.arccos((k3**2 + k2**2 - k1**2) / ( 2 * k3 * k2 )) 
        theta3 = np.pi - theta2 - theta
        t2 = k3/k2
        t3 = k1/k3
        res =  2 *  (p_min(k1)*p_min(k2) * K(k1,t,theta)
                    +p_min(k3)*p_min(k1) * K(k3,t3,theta3) 
                    +p_min(k2)*p_min(k3) * K(k2,t2,theta2))/(p_min(k1)*p_min(k2)+p_min(k3)*p_min(k1)+p_min(k2)*p_min(k3))
        return res
    
    def Q_approx(k1,t,theta):
        k2 = t * k1
        phi = np.pi - theta
        k3 = np.sqrt((k2*np.sin(phi))**2 + (k2*np.cos(phi)+k1)**2)    
        theta2 = np.arccos((k3**2 + k2**2 - k1**2) / ( 2 * k3 * k2 )) 
        theta3 = np.pi - theta2 - theta        
        t2 = k3/k2
        t3 = k1/k3
        res =    (2 *p_approx(k1)*p_approx(k2) * K(k1,t,theta)
                    +2 *p_approx(k3)*p_approx(k1) * K(k3,t3,theta3) 
                    +2 *p_approx(k2)*p_approx(k3) * K(k2,t2,theta2))/(p_approx(k1)*p_approx(k2)+p_approx(k3)*p_approx(k1)+p_approx(k2)*p_approx(k3))
        return res
        
    def Bispektrum(k1,t,theta):
        k2 = t * k1
        phi = np.pi - theta
        k3 = np.sqrt((k2*np.sin(phi))**2 + (k2*np.cos(phi)+k1)**2)    
        theta2 = np.arccos((k3**2 + k2**2 - k1**2) / ( 2 * k3 * k2 )) 
        theta3 = np.pi - theta2 - theta
        t2 = k3/k2
        t3 = k1/k3
        res =  (2*p(k1)*p(k2)*K(k1,t,theta) +2*p(k3)*p(k1)*K(k3,t3,theta3) +2*p(k2)*p(k3)*K(k2,t2,theta2))
        return res
    
    
    fig = plt.figure(np.random.randint(0,1110000))
    
    ax = fig.subplots(nrows=2, ncols=2)
    #plt.figure(12312312312123)
    if EdSitter == True:
        fig.suptitle("EdSitter")
    else:
        fig.suptitle("Not EdSitter")
    theta = np.linspace(0,np.pi,50)
    #k1 = 0.03 / h
    
    k_list_test = np.array([0.03,0.1,0.05,0.4]) 
    #print(k_list_test)
    t = 2
    k_i = 0
    for row in ax:
        for col in row:
            k1 = k_list_test[k_i]/h
            #print(k1)
            col.plot(theta,reduceret_B_spektrum(k1,t,theta),label="class P(k)")     
            col.plot(theta,Q_min(k1,t,theta),"-.",label="Med min egne P(k)")
            col.scatter(theta,Q_approx(k1,t,theta),s=2,label="Med approx P(k)")
            col.set_title("$k_1 = %s [h/Mpc] , t = %s$"% (k_list_test[k_i],t))
            k_i += 1
            
            col.set_xlabel("$\\theta$")
            col.set_ylim([0,3.5])
            col.grid()
            col.set_ylabel("$Q(k_1,t,\\theta)$",rotation=90)
    
    plt.subplots_adjust(top=1.2)
    fig.tight_layout(pad=1.2)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    tol = 1e-3
    n = 30
    m = n//5
    mat = np.empty((n,n))
    k1 = 0.15
    t = 2
    for i in range(n):
        for j in range(n):
            t  = 1/2 * ( 1 + i/n)
            theta = np.pi - np.arccos(1/2*( 1 + j/n))
            if 2 * t * np.cos(np.pi - theta) >= 1:
                mat[i,j] =  Bispektrum(k1/h,t,theta)
            else:
                mat[i,j]=None
        #list)
    plt.matshow(mat,interpolation="spline16")
    plt.gca().invert_yaxis()
    plt.gca().xaxis.tick_bottom()
    xtick_numbers = np.array([np.pi - np.arccos(1/2*( 1 + j/n)) for j in range(n)])
    x = np.cos(np.pi-xtick_numbers[::m])
    plt.xticks([i for i in range(n)][::m],np.round(x,1))
    ytick_numbers = np.array([1/2 * ( 1+ j/n) for j in range(n)])[::m]
    plt.yticks([i for i in range(n)][::m],np.round(ytick_numbers,1))
    plt.ylabel("t",rotation=0)
    plt.xlabel("$cos(\pi-\\theta)$")
    plt.title("Matter Bispectrum $k_1=%s [h/Mpc]$" % k1)
    plt.colorbar()
    print("Det er værd at bemærke at min bispektrum er 2 gange større end Johans Bispektrum, hvilket kan fikses ved at fjerne 2 faktoren i Bispektret")
    
    fig = plt.figure(np.random.randint(0,1110000))
    def Bispektrum_med_z(k1,t,theta,zend=0):
        k2 = t * k1
        phi = np.pi - theta
        k3 = np.sqrt((k2*np.sin(phi))**2 + (k2*np.cos(phi)+k1)**2)    
        theta2 = np.arccos((k3**2 + k2**2 - k1**2) / ( 2 * k3 * k2 )) 
        theta3 = np.pi - theta2 - theta
        t2 = k3/k2
        
        t3 = k1/k3
        delta_approx = np.empty(len(kvec))
        for kind,k in enumerate(kvec): 
            delta_class = pts[kind]["delta_cdm"]
            tau_class = pts[kind]["tau [Mpc]"]
            a_class = pts[kind]["a"]
            interp_tau_from_a = scipy.interpolate.interp1d(a_class,tau_class,kind=interpolation_type)
            delta_interp = scipy.interpolate.interp1d(tau_class,delta_class,kind=interpolation_type)
            a0 = 1/(1+z)
            T0 = float(interp_tau_from_a(a0))
            delta0 = delta_interp(T0) * 1/(1+zend)
            #print(delta0)

            delta_approx[kind] = delta0/a0 

        delta_app =  scipy.interpolate.interp1d(kvec,delta_approx)
        #print("hej")
        #print(f"{delta_app(1)=}")
        p = lambda k: 2*np.pi**2 * delta_app(k)** 2 *  prim_interp(k) * h**3  *k**(-3)    
        #print(f"{p(1)=}")
        res =  (2*p(k1)*p(k2)*K(k1,t,theta) +2*p(k3)*p(k1)*K(k3,t3,theta3) +2*p(k2)*p(k3)*K(k2,t2,theta2))
        return res
    
    k = np.logspace(-2,np.log10(2),20) / h
    #print(k)
    #print(f"{Bispektrum(1,1,np.pi/3)=}")
    fig = plt.figure(np.random.randint(0,1110000))
    plt.title(" Equilateral Matter Bispectrum for forskellige a-værdier")
    plt.loglog(k,k**6 * Bispektrum_med_z(k,1,np.pi/3),label="a=1")
    plt.loglog(k,k**6 * Bispektrum_med_z(k,1,np.pi/3,zend=1),label="a=0.5")
    plt.loglog(k,k**6 * Bispektrum_med_z(k,1,np.pi/3,zend=9),label="a=0.1")
    plt.ylim([1e-17,1e7])
    plt.legend()
    plt.xlabel("k [h/Mpc]")
    plt.ylabel("$k^6 B(k_1,\\theta,t)$")
    fig = plt.figure(np.random.randint(0,1110000))
    plt.loglog(k,Bispektrum(k,1,np.pi/3))
    plt.loglog(k,Bispektrum(k,2,np.pi/3))
    plt.loglog(k,Bispektrum(k,3,np.pi/3))
    #plt.loglog(k,Bispektrum(k,4,np.pi/3))
    plt.xlabel("k [h/Mpc]")
    
    plt.ylabel("$B(k_1,\\theta,t)$")
    
    fig = plt.figure(np.random.randint(0,1110000))
    k = np.logspace(-2,np.log10(2),20) / h
    plt.loglog(k,K(k,t,theta))
    
kvec1=list(np.logspace(-4,1,10))
TEST(kvec = kvec1,EdSitter=True)
#TEST(kvec = kvec1,EdSitter=False)

NameError: name 'inter_tid_from_a' is not defined